In [314]:
import os
import traceback

import numpy as np
import pandas as pd
import re
from rouge_score import rouge_scorer
from typing import Optional
from sklearn.metrics import classification_report

In [315]:
results_overview = dict()

In [316]:
root_path = os.path.dirname(os.path.dirname(os.getcwd()))

In [317]:
job_id = "5494445"
model_name = "llama-2"
model_size = "13B"
results_path = os.path.join(root_path, "results_inference", model_name, model_size, job_id)
test_set = pd.read_csv(os.path.join(results_path, "test.csv"))
print(test_set.shape[0])
print(test_set.columns)

1738
Index(['sample_index', 'completion', 'prompt', 'source', 'input_prompt',
       'inferred_completion'],
      dtype='object')


In [318]:
def has_python_script(text: str) -> bool:
    """There is a complete or incomplete script in the LLM completion"""
    return (not pd.isna(text)) and ("```python" in text or "```" in text)

def has_non_python_script(completion: str):
    """CodeLlama gives Golang scripts, detect those by some common keywords"""
    return (not pd.isna(completion)) and ("package main" in completion or "Println" in completion or ":=" in completion)

def extract_script(text: str) -> Optional[str | None]:
    """Extract Python scripts from the completion"""
    if pd.isna(text):
        return np.nan
    code_extraction_regex, code_extraction_regex_backup = r"```python([^`]+)```", r"```([^`]+)```"
    scripts = re.findall(code_extraction_regex, text)
    if not scripts:
        # rare case: max_tokens reached before "```" indicator is completely returned by LLM
        scripts = re.findall(code_extraction_regex_backup, text)
        if not scripts:
            return np.nan
    return scripts[0].replace("`", "").replace("python", "")  # "\n".join(scripts)

### Any empty completions?

In [319]:
empty_completions = test_set[(test_set["inferred_completion"].isna()) | (test_set["inferred_completion"].isin(["", " "]))].shape[0]
results_overview["empty_completions"] = empty_completions
empty_completions

0

### How many completions with Python scripts?

In [320]:
test_set["has_script"] = test_set["inferred_completion"].apply(lambda completion: has_python_script(completion))

python_scripts = test_set[test_set["has_script"] == True].shape[0]
results_overview["python_scripts"] = python_scripts
python_scripts

1416

### How many completions with Golang scripts?

In [321]:
test_set["has_non_python_script"] = test_set["inferred_completion"].apply(lambda completion: has_non_python_script(completion))

non_python_scripts = test_set[test_set["has_non_python_script"] == True].shape[0]
results_overview["non_python_scripts"] = non_python_scripts
non_python_scripts

0

### Check there cannot be both Python and non-Python scripts in one completion

In [322]:
test_set[(test_set["has_script"]==True) & (test_set["has_non_python_script"]==True)].shape[0]

0

#### Per dataset, what is the split between generated (complete or not) scripts and missing scripts?

In [323]:
test_set.groupby(["source", "has_script"]).count()["sample_index"]

source      has_script
awpnli      False           55
            True           125
newsnli     False           42
            True           200
redditnli   False           12
            True            50
rtequant    False            3
            True            38
stresstest  False          210
            True          1003
Name: sample_index, dtype: int64

#### Inspect some of the identified Python scripts

In [324]:
test_set["inferred_script"] = test_set["inferred_completion"].apply(lambda completion: extract_script(completion))

In [325]:
test_set[test_set["has_script"] == True][["completion", "inferred_completion", "inferred_script"]].head(5)

,completion,inferred_completion,inferred_script
0,```python\nmax_gloves_premise = 60\ngloves_hyp...,$}}%\n\n```\n# Extract the quantities from the...,\n# Extract the quantities from the premise an...
1,```python\nstretch_time_premise = 10\nmax_stre...,$}}%python\n\n### Input:\nPremise: After 10 mi...,\nstop_time_premise = 10\nless_than_20_minutes...
2,```python\nincome_spend_on_petrol_premise = 0....,```python\nincome_premise = 100 # assume the...,\nincome_premise = 100 # assume the premise is...
4,```python\nbutter_kg_premise = 27\nbutter_kg_h...,$}}% python\n\n# Extract quantities from input...,NaN
5,```python\nmark_boxes_sold_premise = 6\nann_bo...,$}}%\n\n```\n# Extract quantities from the pre...,\n# Extract quantities from the premise and hy...


### We notice that some scripts only return a boolean value, but there is no reasoning/math logic to infer it, so these scripts should be skipped during the evaluation

In [326]:
def returns_only_label(script: str):
    """Detect Python scripts that return directly a label, with no reasoning"""
    if pd.isna(script):
        return False
    script = script.replace("\n", "").strip()
    return script == "True" or script == "False"

In [327]:
test_set["only_label"] = test_set["inferred_script"].apply(lambda script: returns_only_label(script))
label_only = test_set[test_set["only_label"] == True].shape[0]
results_overview["label_only"] = label_only
label_only

5

#### For how many samples the `max_new_tokens` limit was exceeded before the full script could not be outputted?

In [328]:
def incomplete_script(has_script: bool, script: Optional[str | None]) -> bool:
    return has_script and pd.isna(script)

test_set["tokens_exceeded"] = test_set.apply(lambda row: incomplete_script(row["has_script"], row["inferred_script"]), axis=1)
tokens_exceeded = test_set[test_set["tokens_exceeded"]==True].shape[0]
results_overview["tokens_exceeded"] = tokens_exceeded
tokens_exceeded

100

In [329]:
test_set["expected_script"] = test_set["completion"].str.replace("```", "")
test_set["expected_script"] = test_set["expected_script"].str.replace("python", "")

### COMPUTE SIMILARITY BETWEEN TARGET AND INFERRED COMPLETION

In [330]:
# Your generated text and the reference text
generated_text = "The weather outside is so good today!"
reference_text = "Today we have such good weather outside!"

#### ROUGE metric

In [331]:
# Create a scorer object
scorer_1 = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
scorer_2 = rouge_scorer.RougeScorer(['rouge2'], use_stemmer=True)
scorer_L = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def rouge1_score(reference_text: str, generated_text: str, compute_score: bool):
    if not (pd.isna(reference_text) or pd.isna(generated_text)) and compute_score:
        return round(scorer_1.score(reference_text, generated_text)['rouge1'].fmeasure, 4)
    return np.nan

def rouge2_score(reference_text: str, generated_text: str, compute_score: bool):
    if not (pd.isna(reference_text) or pd.isna(generated_text)) and compute_score:
        return round(scorer_2.score(reference_text, generated_text)['rouge2'].fmeasure, 4)
    return np.nan

def rougeL_score(reference_text: str, generated_text: str, compute_score: bool):
    if not (pd.isna(reference_text) or pd.isna(generated_text)) and compute_score:
        return round(scorer_L.score(reference_text, generated_text)['rougeL'].fmeasure, 4)
    return np.nan

In [332]:
print(rouge1_score(reference_text, generated_text, True))
print(rouge2_score(reference_text, generated_text, True))
print(rougeL_score(reference_text, generated_text, True))

0.5714
0.1667
0.2857


#### METEOR metric

In [333]:
import nltk
from nltk.translate.meteor_score import single_meteor_score

# Ensure that NLTK's tokenizers and taggers are downloaded
nltk.download('wordnet')
nltk.download('omw-1.4')

def meteor_score(reference_text: str, generated_text: str, compute_score: bool):
    if not (pd.isna(reference_text) or pd.isna(generated_text)) and compute_score:
        return round(single_meteor_score(reference_text.split(" "), generated_text.split(" ")), 4)
    return np.nan

print(meteor_score(reference_text, generated_text, True))

0.1429


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ioanamazilu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ioanamazilu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [334]:
# import nltk
# from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu
# from tree_sitter import Language, Parser
#
# # Load your programming language grammar, assuming Python here
# # Language.build_library(
# #   'build/my-languages.so',
# #   ['python']
# # )
# PY_LANGUAGE = Language('parser/my-languages.so', 'python')
# parser = Parser()
# parser.set_language(PY_LANGUAGE)
#
# def get_tokens(code):
#     """Extract tokens from source code using tree-sitter."""
#     tree = parser.parse(bytes(code, "utf8"))
#     root_node = tree.root_node
#     tokens = [node.text.decode('utf8') for node in root_node.walk() if node.type == 'identifier']
#     return tokens
#
# # Example codes
# generated_code = "def add(a, b):\n    return a + b"
# reference_code = ["def add(num1, num2):\n    return num1 + num2"]
#
# # Tokenize codes
# tokenized_generated_code = get_tokens(generated_code)
# tokenized_reference_code = [get_tokens(ref) for ref in reference_code]
#
# # Compute BLEU score
# chencherry = SmoothingFunction()
# bleu_score = corpus_bleu(tokenized_reference_code, [tokenized_generated_code], smoothing_function=chencherry.method1)
#
# print(f"CodeBLEU (simplified, BLEU part only) score: {bleu_score}")

#### CHRF score

In [335]:
import nltk
from nltk.translate.chrf_score import sentence_chrf

# Ensure that NLTK's tokenizers and data are available
nltk.download('punkt')

def chrf_score(reference_text: str, generated_text: str, compute_score: bool):
    if not (pd.isna(reference_text) or pd.isna(generated_text)) and compute_score:
        return round(sentence_chrf(reference_text, generated_text), 4)
    return np.nan

print(f"CHRF score: {chrf_score(reference_text, generated_text, True)}")

CHRF score: 0.4935


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ioanamazilu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Compare inferred and expected scripts only for valid Python scripts

In [336]:
def evaluate_script(row):
    """Evaluate a script only if it is a complete Python script that does not return a label directly """
    return not row["tokens_exceeded"] and not row["only_label"] and row["has_script"]

test_set["evaluate_script"] = test_set.apply(lambda row: evaluate_script(row), axis=1)
evaluated_scripts = test_set[test_set["evaluate_script"]==True].shape[0]
results_overview["evaluated_scripts"] = evaluated_scripts
evaluated_scripts

1311

In [337]:
test_set["rouge1_score"] = test_set.apply(lambda row: rouge1_score(row["expected_script"], row["inferred_script"], row["evaluate_script"]), axis=1)
test_set["rouge2_score"] = test_set.apply(lambda row: rouge2_score(row["expected_script"], row["inferred_script"], row["evaluate_script"]), axis=1)
test_set["rougeL_score"] = test_set.apply(lambda row: rougeL_score(row["expected_script"], row["inferred_script"], row["evaluate_script"]), axis=1)
test_set["meteor_score"] = test_set.apply(lambda row: meteor_score(row["expected_script"], row["inferred_script"], row["evaluate_script"]), axis=1)
test_set["chrf_score"] = test_set.apply(lambda row: chrf_score(row["expected_script"], row["inferred_script"], row["evaluate_script"]), axis=1)

In [338]:
score_columns = [col for col in test_set.columns if "_score" in col]
test_set[score_columns].aggregate(["mean", "std"])

,rouge1_score,rouge2_score,rougeL_score,meteor_score,chrf_score
mean,0.585834,0.336857,0.459307,0.479989,0.569066
std,0.112145,0.122715,0.118895,0.134224,0.116864


## Write the inferred scripts to python files for QNLI classification evaluation & code quality evaluation

In [339]:
test_set["evaluate_script"].value_counts()

evaluate_script
True     1311
False     427
Name: count, dtype: int64

In [340]:
dataset_name_mapping = {
    "stresstest": "StressTest",
    "rtequant": "RTE_Quant",
    "awpnli": "AWPNLI",
    "newsnli": "NewsNLI",
    "redditnli": "RedditNLI"
}

for _, dataset in dataset_name_mapping.items():
    os.makedirs(os.path.join(results_path, "scripts", dataset), exist_ok=True)

def write_script_to_file(sample_index: str, source_dataset: str, script: Optional[str | None], evaluate_script: bool):
    if not evaluate_script:
        return False
    try:
        with open(os.path.join(results_path, "scripts", dataset_name_mapping[source_dataset], f"sample_{sample_index}.py"), 'x') as f:
            f.write(script)
        return True
    except:
        print(traceback.print_exc())
        return False

In [341]:
test_set["script_to_file"] = test_set.apply(lambda row: write_script_to_file(row["sample_index"], row["source"], row["inferred_script"], row["evaluate_script"]), axis=1)

In [342]:
test_set[test_set["script_to_file"] == True].shape[0]

1311

In [343]:
from src.utils import run_script
from src.dataset_validation import validate_code_quality

count = 0
for idx, row in test_set.iterrows():
    print(idx)
    qualified_for_classification = getattr(row, "evaluate_script")
    if qualified_for_classification:
        count += 1
        sample_index, dataset_key = getattr(row, "sample_index"), getattr(row, "source")
        dataset = dataset_name_mapping[dataset_key]
        script_path = os.path.join(results_path, "scripts", dataset, f"sample_{sample_index}.py")
        label, error_message = run_script(script_path)
        test_set.loc[idx, "inferred_label"] = label
        test_set.loc[idx, "error_message"] = error_message

        code_quality_scores, code_quality_resolutions = validate_code_quality(script_path)
        for score_key, score in code_quality_scores.items():
            test_set.loc[idx, score_key] = score
        test_set.loc[idx, "cc_resolutions"] = "; ".join(code_quality_resolutions)

print(f"Evaluated {count} scripts.")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [344]:
labels_obtained = test_set[test_set["inferred_label"].isin(["entailment", "contradiction", "neutral"])].shape[0]
results_overview["labels_obtained"] = labels_obtained
labels_obtained

642

In [345]:
code_quality_scores = ['readability', 'document_size', 'redundancy_check', 'function_size']

test_set[code_quality_scores].aggregate(["mean", "std"])

,readability,document_size,redundancy_check,function_size
mean,94.633036,75.539283,84.229939,54.271548
std,14.263587,23.349350,4.500286,13.981431


## View results overview

In [346]:
results_overview

{'empty_completions': 0,
 'python_scripts': 1416,
 'non_python_scripts': 0,
 'label_only': 5,
 'tokens_exceeded': 100,
 'evaluated_scripts': 1311,
 'labels_obtained': 642}

## QNLI results evaluation